In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE273630"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE273630"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE273630.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE273630.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE273630.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dopamine-regulated biomarkers in peripheral blood of HIV+ Methamphetamine users"
!Series_summary	"HIV and Methamphetamine study - Translational Methamphetamine AIDS Research Center - Dopamine-regulated inflammatory biomarkers"
!Series_summary	"A digital transcript panel was custom-made based on Hs_NeuroPath_v1 (Nanostring) to accommodate dopamine-regulated inflammatory genes that were previously identified in vitro, and hypothesized to cluster HIV+ Methamphetamine users."
!Series_overall_design	"Specimens were peripheral blood leukocytes isolated from participants that included adults enrolled by NIH-funded studies at the University of California San Diego’s HIV Neurobehavioral Research Program (HNRP) and Translational Methamphetamine Research Center (TMARC) under informed consent and approved protocols. The subset of PWH and PWoH selected for this study were by design males, between 35 – 44 years old, due to cohort characteristics and to increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Check gene expression data availability
is_gene_available = False  # Based on information, this seems to be a targeted panel of dopamine-regulated genes, not full gene expression

# Check trait, age, and gender data availability
# From the background information, we can infer:
# - Participants are divided based on HIV and Meth use status
# - All participants are males between 35-44 years old

trait_row = None  # The trait (schizophrenia) is not available in this dataset
age_row = None    # Age is constant (35-44 years) so not useful for association study
gender_row = None # Gender is constant (all males) so not useful for association study

# Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    # Not needed as trait data isn't available
    return None

def convert_age(value: str) -> Optional[float]:
    # Not needed as age data isn't available
    return None

def convert_gender(value: str) -> Optional[int]:
    # Not needed as gender data isn't available
    return None

# Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# We skip clinical feature extraction since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE273630/GSE273630_series_matrix.txt.gz
Gene data shape: (780, 99)
First 20 gene/probe identifiers:
Index(['ABAT', 'ABL1', 'ACAA1', 'ACHE', 'ACIN1', 'ACTN1', 'ACVRL1', 'ADAM10',
       'ADCY5', 'ADCY8', 'ADCY9', 'ADCYAP1', 'ADORA1', 'ADORA2A', 'ADRA2A',
       'ADRB2', 'AGER', 'AIF1', 'AKT1', 'AKT1S1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed, these appear to be human gene symbols.
# They follow standard HGNC gene symbol nomenclature (e.g., ABAT, ABL1, ACAA1).
# These are official gene symbols, not probe IDs which would typically be numerical or have a different format.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no schizophrenia data available),
# we cannot create proper clinical data for this trait
print("Clinical data for the trait 'Schizophrenia' is not available in this dataset.")

# The dataset is about HIV and Methamphetamine use, not Schizophrenia
print("This dataset (GSE273630) contains information about HIV and Methamphetamine use, not Schizophrenia.")

# Create an empty DataFrame to represent the missing clinical data
empty_df = pd.DataFrame()

# 5. Validate and save cohort information - mark as not usable for the trait
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # No trait data for Schizophrenia
    is_biased=True,  # Set to True to indicate an unusable dataset for this trait
    df=empty_df,  # Empty DataFrame as there's no linked data
    note="Dataset contains gene expression from HIV and Methamphetamine users. No Schizophrenia data available."
)

print("Dataset is not usable for Schizophrenia analysis. No linked data file saved.")

Gene data shape after normalization: (780, 99)
Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE273630.csv
Clinical data for the trait 'Schizophrenia' is not available in this dataset.
This dataset (GSE273630) contains information about HIV and Methamphetamine use, not Schizophrenia.
Abnormality detected in the cohort: GSE273630. Preprocessing failed.
Dataset is not usable for Schizophrenia analysis. No linked data file saved.
